## Data Ingestion Notebook

In [1]:
import os 
os.chdir("../")

In [2]:
pwd

'/home/jvdboss/workspace/ML_DL/image_classification/Lung_Xray_Classifier'

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path
    train_path: Path
    test_path: Path
    params_test_ratio: float

@dataclass
class DataIngestionArtifacts:
    ingested_train_dir : Path
    ingested_test_dir: Path

In [4]:
from xray.constants import *
from xray.utils import read_yaml, create_directories

In [5]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir,
            train_path=config.train_path,
            test_path=config.test_path,
            params_test_ratio=self.params.TEST_RATIO,
        )

        return data_ingestion_config

In [6]:
import os
import urllib.request as request
from zipfile import ZipFile
import numpy as np
import shutil
import random

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )

    def _get_updated_list_of_files(self, list_of_files):
        return [f for f in list_of_files if f.endswith(".jpeg") and ("NORMAL" in f or "PNEUMONIA" in f)]

    def _preprocess(self, zf: ZipFile, f: str, working_dir: str):
        target_filepath = os.path.join(working_dir, f)
        if not os.path.exists(target_filepath):
            zf.extract(f, working_dir)
        
        if os.path.getsize(target_filepath) == 0:
            os.remove(target_filepath)
            
    def unzip_and_clean(self):
        
        with ZipFile(file=self.config.local_data_file, mode="r") as zf:
            #print(self.config.local_data_file)
            list_of_files = zf.namelist()
            updated_list_of_files = self._get_updated_list_of_files(list_of_files)
            # 1 print(updated_list_of_files)
            for f in updated_list_of_files:
                self._preprocess(zf, f, self.config.unzip_dir)
    
    def train_test_split(self):
        """This function would split the raw data into train and test folder
        """
        try:
            # 1. make train and test folder 
            unzip_images = self.config.unzip_dir
            train_path = self.config.train_path
            test_path = self.config.test_path
            
            #params.yaml
            test_ratio = self.config.params_test_ratio
            # 1. make train and test folder 
            train_path = os.path.join(os.getcwd(), unzip_images, train_path)
            test_path = os.path.join(os.getcwd(), unzip_images, test_path)
            print(train_path)
            classes_dir = ['NORMAL', 'PNEUMONIA']

            for cls in classes_dir:
                os.makedirs(os.path.join(train_path, cls), exist_ok= True)
                os.makedirs(os.path.join(test_path, cls), exist_ok=True)
            
            # 2. Split the raw data
            raw_data_path = os.path.join(os.getcwd(), unzip_images, 'chest_xray')
            for cls in classes_dir:
                allFileNames = os.listdir(os.path.join(raw_data_path, cls))
                #print(allFileNames)
                np.random.shuffle(allFileNames)
                train_FileNames, test_FileNames = np.split(np.array(allFileNames),
                                    [int(len(allFileNames)* (1 - test_ratio))])

                train_FileNames = [os.path.join(raw_data_path, cls, name) for name in train_FileNames.tolist()]
                test_FileNames = [os.path.join(raw_data_path, cls, name) for name in test_FileNames.tolist()]

                #print(train_FileNames)
                for name in train_FileNames:
                    shutil.copy(name, os.path.join(train_path, cls))

                for name in test_FileNames:
                    shutil.copy(name, os.path.join(test_path, cls))
                
                #print(test_FileNames)
            data_ingestion_artifact = DataIngestionArtifacts(ingested_train_dir=train_path,
                                                             ingested_test_dir= test_path)
            return data_ingestion_artifact

        except Exception as e:
            raise e

    def remove_raw_data_dir(self):
        
        shutil.rmtree(os.path.join(self.config.unzip_dir,'chest_xray'), ignore_errors=True)
        shutil.rmtree(os.path.join(self.config.local_data_file), ignore_errors=True)

    def run_data_ingestion(self):
        self.download_file()
        self.unzip_and_clean()
        data_ingestion_artifact = self.train_test_split()
#         self.train_test_split()
        self.remove_raw_data_dir()
        return data_ingestion_artifact
        

In [7]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    artifact=data_ingestion.run_data_ingestion()
except Exception as e:
    raise e

[2022-10-15 17:24:13,750: INFO: common]: yaml file: configs/config.yaml loaded successfully
[2022-10-15 17:24:13,755: INFO: common]: yaml file: params.yaml loaded successfully
[2022-10-15 17:24:13,756: INFO: common]: created directory at: artifacts
[2022-10-15 17:24:13,757: INFO: common]: created directory at: artifacts/data_ingestion
/home/jvdboss/workspace/ML_DL/image_classification/Lung_Xray_Classifier/artifacts/data_ingestion/train
['NORMAL-229022-0001.jpeg', 'NORMAL-239549-0001.jpeg', 'NORMAL-767940-0001.jpeg', 'NORMAL-230877-0001.jpeg', 'NORMAL-1044645-0001.jpeg', 'NORMAL-534646-0001.jpeg', 'NORMAL-170867-0002.jpeg', 'NORMAL-678492-0002.jpeg', 'NORMAL-824917-0001.jpeg', 'NORMAL-233341-0001.jpeg', 'NORMAL-202916-0001.jpeg', 'NORMAL-170867-0001.jpeg', 'NORMAL-447225-0001.jpeg', 'NORMAL-462175-0002.jpeg', 'NORMAL-145240-0001.jpeg', 'NORMAL-284113-0001.jpeg', 'NORMAL-974509-0002.jpeg', 'NORMAL-1041364-0001.jpeg', 'NORMAL-705219-0001.jpeg', 'NORMAL-930887-0001.jpeg', 'NORMAL-712032-00

In [8]:
artifact

DataIngestionArtifacts(ingested_train_dir='/home/jvdboss/workspace/ML_DL/image_classification/Lung_Xray_Classifier/artifacts/data_ingestion/train', ingested_test_dir='/home/jvdboss/workspace/ML_DL/image_classification/Lung_Xray_Classifier/artifacts/data_ingestion/test')